# Regressão Linear 

![](https://miro.medium.com/max/482/1*D6s2K1y7kjE14swcgITB1w.png)

### **Você sabe quanto vale o imóvel em que você mora? Como você faria para predizer esse valor?**

![](https://media.giphy.com/media/eoje0OqGEx5Oo/giphy.gif)

O que _**prever o valor de um imóvel**_ tem em comum com prever...
- a renda de uma pessoa
- a quantidade de vendas de uma loja
- o número de leads de uma empresa

?

Existem diversos algoritmos para resolver esses problemas de regressão. Um deles é a Regressão Linear:

<tr>
    <td> <img src="https://image.slidesharecdn.com/8-1209490505240696-9/95/multiple-linear-regression-16-638.jpg?cb=1489720634" width="450"/></td>,
    <td> <img src="https://cdn-images-1.medium.com/max/1600/1*eeIvlwkMNG1wSmj3FR6M2g.gif" width="450" /> </td>,
</tr>"
    
    

## Exemplo - predição do valor de uma corrida de táxi

<img src="https://media.giphy.com/media/eKt8k3KQLRSFbP4v3f/giphy.gif" width="300"/>

Vamos usar a regressão linear para predizer o valor a ser pago em uma corrida de táxi.

## Bibliotecas auxiliares

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

O nosso conjunto de dados contém as corridas de táxi da cidade de New York:

In [ ]:
df = pd.read_csv("dataset.csv")

#### Descrição do conjunto de dados

|Campo            | Descrição                                                  |
|-----------------|------------------------------------------------------------|
|key              |chave única indicando cada exemplo                          |
|**fare_amount**  |valor, em doláres, indicando o valor pago na corrida de táxi|
|pickup_datetime  |timestamp indicando quando a corrida começou                |
|pickup_longitude |longitude na qual a corrida de táxi começou                 |
|pickup_latitude  |latitude na qual a corrida de táxi começou                  |
|dropoff_longitude|longitude na qual a corrida de táxi acabou                  |
|dropoff_latitude |latitude na qual a corrida de táxi acabou                   |
|passenger_count  |número de passageiros                                       |

In [ ]:
# Vamos explorar nosso conjunto de dados. Podemos usar o head, shape, info, etc.
# TODO

Apenas observando essas features, quais vocês acham que podem nos ajudar a predizer o valor da corrida? Podemos criar alguma feature nova a partir delas?

## Análise exploratória e Feature engineering

### Valores faltantes

In [ ]:
# Remoção de valores faltantes
# TODO

### Distribuição das features

In [ ]:
# Dar um describe no nosso conjunto de dados
# TODO

### Análise das features (e do target)

**`fare_amount`**

Começando pelo target, podemos ver a sua distribuição:

In [ ]:
# Plotar o boxplot do target
# TODO

In [ ]:
# Remover os exemplos com valores discrepantes
# TODO

Podemos ver as distribuições novamente:

In [ ]:
# Plotar as distribuições novamente
# Podemos fazer isso usando o describe, boxplot e distplot
# TODO

**`pickup_datetime`**

Essa feature que indica quando a corrida começou precisa ser convertida para uma data para que possamos trabalhar com ela:

In [ ]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
df.info()

Mesmo com essa conversão, ainda não conseguimos usar essa informação do jeito que está. Podemos quebrá-la em várias features diferentes:

In [ ]:
# horário
df['pickup_hour']=df['pickup_datetime'].apply(lambda x:x.hour)
# dia da semana (0 é segunda e 6 é domingo)
df['pickup_day_of_week']=df['pickup_datetime'].apply(lambda x:x.weekday())
# dia
df['pickup_day']=df['pickup_datetime'].apply(lambda x:x.day)
# mês
df['pickup_month']=df['pickup_datetime'].apply(lambda x:x.month)
# ano
df['pickup_year']=df['pickup_datetime'].apply(lambda x:x.year)

**Horário**

In [ ]:
# Countplot dos horários
# TODO

In [ ]:
# Catplot dos horários vs o target (fazer o tipo "bar")
# TODO

**Dia da semana**

In [ ]:
# Vamos fazer a conversão dos dias da semana para facilitar a leitura
def convert_day_of_week(day_of_week):
    day_dict={0:'Segunda',1:'Terça',2:'Quarta',3:'Quinta',4:'Sexta',5:'Sábado',6:'Domingo'}
    return day_dict[day_of_week]

In [ ]:
df['pickup_day_of_week_name']=df['pickup_day_of_week'].apply(lambda x:convert_day_of_week(x))

In [ ]:
week_day_name = ['Segunda','Terça','Quarta','Quinta','Sexta','Sábado','Domingo']

In [ ]:
# Countplot dos dias da semana
# TODO

In [ ]:
# Catplot dos dias da semana vs o target (fazer o tipo "bar")
# TODO

**Dia**

In [ ]:
# Countplot dos dias
# TODO

In [ ]:
# Catplot dos dias vs o target (fazer o tipo "bar")
# TODO

**Mês**

In [ ]:
# Countplot dos meses
# TODO

In [ ]:
# Catplot dos meses vs o target (fazer o tipo "bar")
# TODO

**Ano**

In [ ]:
# Countplot dos anos
# TODO

In [ ]:
# Catplot dos anos vs o target (fazer o tipo "bar")
# TODO

**`latitude e longitude`**

In [ ]:
df[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']].describe()

As coordenadas de latitude variam apenas entre -90 e 90, enquanto que a longitude varia apenas entre -180 e 180. 
Podemos excluir todos os exemplos cujas longitudes e latitudes estão fora desse intervalo:

In [ ]:
print(f'Tamanho conjunto de dados (antes da remoção de valores faltantes): {len(df)}')
df = df.loc[(df.pickup_latitude>=-90) & (df.pickup_latitude<=90)]
df = df.loc[(df.pickup_longitude>=-180) & (df.pickup_longitude<=180)]
df = df.loc[(df.dropoff_latitude>=-90) & (df.dropoff_latitude<=90)]
df = df.loc[(df.dropoff_longitude>=-180) & (df.dropoff_longitude<=180)]
print(f'Tamanho conjunto de dados (depois da remoção de valores faltantes): {len(df)}')

In [ ]:
# Dar o describe novamente
# TODO

**Pontos de origem da corrida**

In [ ]:
df.plot(kind='scatter', x='pickup_longitude', y='pickup_latitude',
                color='green', 
                s=.02, alpha=.6)
plt.title("Pickups");

Conforme podemos notar no gráfico acima, parece que algumas corridas ainda são outliers, visto que estão muito distantes das demais, um indício de que essas viagens não tiveram origem na cidade de New York. Vamos remover os casos de pontos que não estão no intervalo abaixo:

In [ ]:
long_interval = (-74.04, -73.75)
lat_interval = (40.63, 40.88)

In [ ]:
df.plot(kind='scatter', x='pickup_longitude', y='pickup_latitude',
                color='green', 
                s=.02, alpha=.6)
plt.title("Origens")
plt.ylim(lat_interval)
plt.xlim(long_interval);

Agora conseguimos visualizar Manhattan :) 

Vamos então filtrar novamente as longitudes e latitudes para considerar somente esses intervalos:

In [ ]:
# Fazer a remoção dos exemplos cujas longitudes e latitudes não estão no intervalo acima
# TODO

**Pontos de destino da corrida**

In [ ]:
# Fazer o mesmo plot para os pontos de destino das corridas
# TODO

In [ ]:
# Fazer a remoção dos outros outliers
# TODO

Além dessa limpeza de outliers, também podemos criar novas variáveis, como as de diferenças absolutas entre as latitudes e as longitudes de origem e destino das corridas:

In [ ]:
# Atribuir a diferença absoluta entre as latitudes e as longitudes nos atributos abaixo
# TODO
df['abs_diff_longitude'] =
df['abs_diff_latitude'] = 

In [ ]:
# Dar um describe nessas novas features
# TODO

**`passenger_count`**

In [ ]:
# Verificar os outliers e removê-los ou substituí-los
# TODO

In [ ]:
# Contar os casos de outliers e removê-los do nosso conjunto de dados
# TODO

In [ ]:
# Countplot do número de passageiros
# TODO

In [ ]:
# Catplot do número de passageiros vs o target (fazer o tipo "bar")
# TODO

### Correlação

In [ ]:
# TODO
# Gerar a matriz de correlação entre as features

#### Pronto, vamos parar por aqui com a análise exploratória e feature engineering hoje!

O que mais poderíamos ter feito?

### Regressão Linear Simples

Vamos criar a nossa primeira regressão linear escolhendo a variável que achamos que será a mais relevante. Para criar esse modelo, podemos usar a implementação da regressão linear existente no [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html):

In [ ]:
# import da regressão linear
from sklearn.linear_model import LinearRegression

In [ ]:
# Fazer um scatterplot com a feature que escolhemos
# TODO

In [ ]:
# Ver a correlação entre o target e a feature
# TODO

#### Separação do conjunto de treino e teste

In [ ]:
# Colocar o nosso target na variável Y
# Colocar o restante das features na variável X
# TODO

In [ ]:
# Separar nosso conjunto de dados entre treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)

#### Treinamento do modelo

In [ ]:
# instancia a regressão linear
lm = LinearRegression()
# como vamos utilizar somente uma feature, vamos passar somente ela para treinar o modelo
lm.fit(?, Y_train)

#### Predição

In [ ]:
# para fazer a predição também só devemos utilizar a feature fornecida no treinamento
Y_pred = lm.predict(?)

In [ ]:
_ = sns.regplot(x=Y_test, y=Y_pred)

Yay, fizemos nossa primeira regressão linear! O legal é que podemos substituir os valores naquela fórmula do início do notebook para entender algumas coisas:

In [ ]:
# vamos pegar de exemplo para analisar o resultado
# TODO

In [ ]:
# aqui conseguimos pegar os coeficientes que compõem a equação da regressão linear
beta=lm.coef_
intercept=lm.intercept_
print(beta)
print(intercept)

In [ ]:
# Criar a fórmula da regressão linear
# TODO

In [ ]:
# Pegar o resultado predito pelo nosso modelo para o exemplo escolhido
# TODO

In [ ]:
# Substituir o valor do exemplo na fórmula e verificar se o resultado é o mesmo que o modelo predisse
# TODO

**Mas será que nosso modelo está certo nessas predições?**

![](https://media.giphy.com/media/26xBI73gWquCBBCDe/giphy.gif)

## Métricas de avaliação

Existem diversas métricas para avaliar o erro da predição do nosso modelo. Hoje iremos ver as seguintes:
- MSE (Mean Squared Error)
- MAE (Mean Absolute Error)
- R²

[Aqui](https://towardsdatascience.com/how-to-select-the-right-evaluation-metric-for-machine-learning-models-part-1-regrression-metrics-3606e25beae0) tem um artigo falando sobre outras métricas de avaliação.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

**`MSE`**

$\frac{1}{N}\sum{(y_{test} - y_{pred})^2}$

O MSE calcula o erro quadrático médio das predições do nosso modelo. Quanto maior o MSE, pior é o modelo.

Essa métrica é útil quando temos valores de target mais incomuns e que seriam interessantes que nosso modelo não errasse. 

Por outro lado, como os erros são penalizados exponencialmente, erros maiores tem um peso maior do que os erros menores. Então, se o nosso modelo faz apenas uma predição muito, muito ruim, o MSE irá elevar esse erro ao quadrado e, com isso, esse erro ficará ainda pior e acabaremos achando que o nosso modelo está performando pior do que realmente está.

Outro ponto é que a escala do MSE não é a mesma do nosso target (visto que os erros são elevados ao quadrado), então sua interpretação fica mais difícil.

In [ ]:
# Calcular o MSE
# TODO

**`MAE`**

$\frac{1}{N}\sum{|y_{test} - y_{pred}|}$

O MAE calcula a média da diferença absoluta entre o valor predito e o valor real. Nesse caso, os erros são penalizados linearmente, ou seja, todos terão o mesmo peso na média.

Sua vantagem perante o MSE é que, como a escala é a mesma do target, é mais fácil interpretá-lo. Além disso, essa métrica não penaliza tanto erros maiores como o MSE e, com isso, o MAE não é tão sensível a outliers como o MSE.

In [ ]:
# Calcular o MAE
# TODO

**`R²`**

$1 - \frac{\frac{1}{N}\sum{(y_{test} - y_{pred})^2}}{\frac{1}{N}\sum{(y_{test} - \bar{y})^2}}$

O R² é uma métrica que varia entre -∞ e 1 e é uma razão que indica o quão bom o nosso modelo está em comparação com um modelo naive que faz a predição com base no valor médio do target. Quanto maior seu valor, melhor é nosso modelo com relação a esse modelo mais simplista.

Um valor de R² negativo significa que nosso modelo é pior do que se tivéssemos feito a predição com o valor da média. Entretanto, só com essa métrica não conseguimos enxergar a magnitude dos erros do nosso modelo.

In [ ]:
# Calcular o R2
# TODO

Hum, mas será que se nós tivéssemos usado a mediana dos valores pagos na corrida teria sido melhor do que criar um modelo de regressão linear? Será que perdemos nosso tempo? :(

In [ ]:
# Calcular o MAE e o R2 se fizéssemos a predição dizendo que a tarifa 
# seria igual a mediana para todos os exemplos
# TODO

Mas será que escolhemos a melhor feature? Vamos tentar criar outro modelo com outra feature que a gente acredite que seja relevante, calculamos seu erro e verificamos se ele está melhor do que o nosso modelo anterior:

In [ ]:
# Treinar o modelo novamente, dessa vez escolhendo uma outra feature
# Fazer a predição no conjunto de teste
# Verificar os erros
# TODO

Vocês acham que faz sentido a gente ficar criando um modelo só com uma feature mesmo ou o ideal seria combinar com outras que temos disponíveis?

## Regressão Linear Múltipla

Sim, nós podemos utilizar utilizar mais de uma feature para fazer a nossa regressão! Quando fazemos isso, é uma **regressão linear múltipla**.

<img src="https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/38_blog_image_1.png" width="400"/>

Vamos então colocar todas as features que temos disponíveis (e que achamos que fazem sentido):

In [ ]:
# Colocar todas as features que iremos utilizar em um array
# TODO

In [ ]:
# Treinar um novo modelo utilizando somente as features listadas no array
# Fazer a predição no conjunto de teste
# Verificar os erros
# TODO

In [ ]:
# Ver os coeficientes
# TODO

Como temos vários coeficientes agora, podemos tentar visualizá-los de uma forma um pouco melhor para entender seus pesos:

In [ ]:
weights = pd.DataFrame(list(zip(columns, lm.coef_.tolist())),columns=['names', 'coefs'])
weights

In [ ]:
# Usar um barplot para visualizar esses coeficientes por feature
# TODO

Será que faz sentido deixar essas features correlacionadas e que já estão presentes em outras features que criamos, como a `pickup_longitude`?

Podemos treinar mais um modelo de regressão linear múltipla, mas dessa vez removendo as features correlacionadas:

In [ ]:
# Treinar o modelo novamente só removendo as features correlacionadas
# Fazer a predição no conjunto de teste
# Verificar os erros
# TODO

### E quais são as vantagens de se fazer um modelo utilizando uma regressão linear?

?

### E quais são as desvantagens?

?

## Próximos passos

![](http://giphygifs.s3.amazonaws.com/media/L6EoLS78pcBag/giphy.gif)

?

<img src="https://media.giphy.com/media/osjgQPWRx3cac/giphy.gif" width="350"/>